In [3]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

num_epochs = 80
batch_size = 100
learning_rate = 0.001

transform = transforms.Compose([
    transforms.Pad(4), # padding값 4
    transforms.RandomHorizontalFlip(), # 랜덤으로 플립
    transforms.RandomCrop(32),
    transforms.ToTensor()])


train_dataset = torchvision.datasets.CIFAR10(root='../data/',
                                             train=True,
                                             transform=transform,
                                             download=True)

test_dataset = torchvision.datasets.CIFAR10(root='../data',
                                            train=False,
                                            transform=transforms.ToTensor())

train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

# 3x3 convolution
def conv3x3(in_channels, out_channels, stride=1):
    return nn.Conv2d(in_channels, out_channels, kernel_size=3,
                     stride=stride, padding=1, bias=False)

# Residual block
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super(ResidualBlock, self).__init__()
        self.conv1 = conv3x3(in_channels, out_channels, stride)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(out_channels, out_channels)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.downsample = downsample
    
    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        if self.downsample:
            residual = self.downsample(x)
        out += residual
        out = self.relu(out)
        return out
    

# ResNet
class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes=10):
        super(ResNet, self).__init__()
        self.in_channels = 16
        self.conv = conv3x3(3, 16)
        self.bn = nn.BatchNorm2d(16)
        self.relu = nn.ReLU(inplace=True)
        self.layer1 = self.make_layer(block, 16, layers[0])
        self.layer2 = self.make_layer(block, 32, layers[1], 2)
        self.layer3 = self.make_layer(block, 64, layers[2], 2)
        self.avg_pool = nn.AvgPool2d(8)
        self.fc = nn.Linear(64, num_classes)
    
    def make_layer(self, block, out_channels, blocks, stride=1):
        downsample = None
        if (stride != 1) or (self.in_channels != out_channels):
            downsample = nn.Sequential(
                conv3x3(self.in_channels, out_channels, stride=stride),
                nn.BatchNorm2d(out_channels))
        
        layers = []
        layers.append(block(self.in_channels, out_channels, stride, downsample))
        self.in_channels = out_channels
        for i in range(1, blocks):
            layers.append(block(out_channels, out_channels))
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.conv(x)
        out = self.bn(out)
        out = self.relu(out)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.avg_pool(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

model = ResNet(ResidualBlock, [2, 2, 2]).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

def update_lr(optimizer, lr):
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

total_step = len(train_loader)
curr_lr = learning_rate
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print(f'Epoch [{(epoch+1)/num_epochs}], Step [{(i+1) / total_step}], Loss: {loss.item():.4f}')
        
    
    if (epoch+1) % 20 == 0:
        curr_lr /= 3
        update_lr(optimizer, curr_lr)
 

    
        

Files already downloaded and verified
Epoch [0.0125], Step [0.2], Loss: 1.5674
Epoch [0.0125], Step [0.4], Loss: 1.5817
Epoch [0.0125], Step [0.6], Loss: 1.4378
Epoch [0.0125], Step [0.8], Loss: 1.2124
Epoch [0.0125], Step [1.0], Loss: 1.1052
Epoch [0.025], Step [0.2], Loss: 1.1314
Epoch [0.025], Step [0.4], Loss: 1.0826
Epoch [0.025], Step [0.6], Loss: 1.1464
Epoch [0.025], Step [0.8], Loss: 1.0837
Epoch [0.025], Step [1.0], Loss: 0.7822
Epoch [0.0375], Step [0.2], Loss: 0.8170
Epoch [0.0375], Step [0.4], Loss: 0.8323
Epoch [0.0375], Step [0.6], Loss: 0.9263
Epoch [0.0375], Step [0.8], Loss: 0.8105
Epoch [0.0375], Step [1.0], Loss: 0.8929
Epoch [0.05], Step [0.2], Loss: 0.8252
Epoch [0.05], Step [0.4], Loss: 0.6929
Epoch [0.05], Step [0.6], Loss: 0.8355
Epoch [0.05], Step [0.8], Loss: 0.6536
Epoch [0.05], Step [1.0], Loss: 0.5889
Epoch [0.0625], Step [0.2], Loss: 0.5957
Epoch [0.0625], Step [0.4], Loss: 0.6041
Epoch [0.0625], Step [0.6], Loss: 0.6497
Epoch [0.0625], Step [0.8], Loss: 

In [6]:
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    print(f"Accuracy of the model on the test images: {(100*correct)/total} %")

torch.save(model.state_dict(), 'resnet.pbtxt')

Accuracy of the model on the test images: 88.24 %
